In [1]:
import bpnet

Using TensorFlow backend.


In [2]:
from pathlib import Path
import numpy as np

In [3]:
import os

In [4]:
model_dir = Path(os.path.expanduser('~/gdrive/projects/chipnexus/data/bpnet-demo/2019-08-06_07-59-29_a8ba0416-3216-42c3-91d2-49fe5da0cc0e'))

In [5]:
ls {model_dir}

bpnet-train.kwargs.json   events.out.tfevents.1565078400.926fa698b19c
config.gin                history.csv
config.gin.json           input-config.gin
contrib.deeplift.h5       log/
contrib.deeplift.null.h5  model.h5
dataspec.yml              modisco/
evaluate.html             note_params.json
evaluate.ipynb            seq_model.pkl
evaluation.valid.json     wandb.json


In [6]:
import gin

gin.enter_interactive_mode()

In [7]:
from bpnet.layers import DilatedConv1D, GlobalAvgPoolFCN
from bpnet.heads import ScalarHead

In [8]:
from bpnet.seqmodel import SeqModel

In [9]:
import keras.backend as K

In [10]:
sm = SeqModel.from_mdir(model_dir)

TF-MoDISco is using the TensorFlow backend.


Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


2019-08-09 18:42:11,185 [WARNING] From /data/ouga/home/ag_gagneur/avsec/.conda/envs/bpnet/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:497: calling conv1d (from tensorflow.python.ops.nn_ops) with data_format=NHWC is deprecated and will be removed in a future version.
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


Instructions for updating:
Use the retry module or similar alternatives.


2019-08-09 18:42:18,671 [WARNING] From /data/ouga/home/ag_gagneur/avsec/.conda/envs/bpnet/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:198: retry (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Use the retry module or similar alternatives.


In [12]:
# remove the session
K.clear_session()

In [13]:
# create training data
seq = np.random.randn(1000, sm.seqlen, 4)
y = np.random.randn(1000)

In [14]:
sm.model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
seq (InputLayer)                (None, 200, 4)       0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 200, 64)      6464        seq[0][0]                        
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 200, 64)      12352       conv1d_1[0][0]                   
__________________________________________________________________________________________________
add_1 (Add)                     (None, 200, 64)      0           conv1d_1[0][0]                   
                                                                 conv1d_2[0][0]                   
__________

## TODO

- create a new model on top

In [15]:
import keras.layers as kl
from keras.models import Model
from bpnet.heads import ScalarHead
from bpnet.layers import GlobalAvgPoolFCN

In [16]:
from copy import deepcopy

In [17]:
from bpnet.layers import DilatedConv1D

In [18]:
sm2 = SeqModel(sm.body,
              heads=[ScalarHead('output',
                                net=GlobalAvgPoolFCN(),
                               )],
              tasks=['output'],
              seqlen=sm.seqlen)

In [20]:
sm2.save("/tmp/bpnet/model.pkl")

In [21]:
sm2.model.layers[1].get_weights()[0][0,0,0]

-0.019280344

In [22]:
# should change
sm2.model.layers[-2].get_weights()[0][0,0]

-0.16226646

In [23]:
sm2.model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
seq (InputLayer)                (None, 200, 4)       0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 200, 64)      6464        seq[0][0]                        
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 200, 64)      12352       conv1d_1[0][0]                   
__________________________________________________________________________________________________
add_1 (Add)                     (None, 200, 64)      0           conv1d_1[0][0]                   
                                                                 conv1d_2[0][0]                   
__________

In [24]:
sm2.save("/tmp/bpnet/model.pkl")

In [25]:
sm2.model.load_weights(model_dir / 'model.h5', by_name=True, skip_mismatch=True)

/data/ouga/home/ag_gagneur/avsec/.conda/envs/bpnet/lib/python3.6/site-packages/keras/engine/saving.py:1140: UserWarning: Skipping loading of weights for layer dense_1 due to mismatch in shape ((64, 1) vs (64, 2)).
  weight_values[i].shape))
/data/ouga/home/ag_gagneur/avsec/.conda/envs/bpnet/lib/python3.6/site-packages/keras/engine/saving.py:1140: UserWarning: Skipping loading of weights for layer dense_1 due to mismatch in shape ((1,) vs (2,)).
  weight_values[i].shape))


In [26]:
sm2.save("/tmp/bpnet/model.pkl")

In [27]:
sm2.model.layers[1].get_weights()[0][0,0,0]

0.028540729

In [23]:
sm.model.layers[1].get_weights()[0][0,0,0]

ValueError: Fetch argument <tf.Variable 'conv1d_1/kernel:0' shape=(25, 4, 64) dtype=float32_ref> cannot be interpreted as a Tensor. (Tensor Tensor("conv1d_1/kernel:0", shape=(25, 4, 64), dtype=float32_ref) is not an element of this graph.)

In [28]:
# should't change from before
sm2.model.layers[-2].get_weights()[0][0,0]

-0.16226646

In [29]:
sm2.body_freeze()

In [30]:
sm2.save("/tmp/bpnet/model.pkl")

In [31]:
sm2.model.layers[1].get_weights()[0][0,0,0]

0.028540729

In [32]:
# should change
sm2.model.layers[-2].get_weights()[0][0,0]

-0.16226646

In [27]:
## TODO - train a model for a bit on some random data. Check if the parameters change

In [33]:
sm2.model.fit(seq, y)

Epoch 1/1
1000/1000 [==============================] - 0s 464us/step - loss: 3.9459


In [33]:
sm2.model.fit(seq, y)

Epoch 1/1
1000/1000 [==============================] - 0s 464us/step - loss: 3.9459


In [34]:
sm2.save("/tmp/bpnet/model.pkl")

In [35]:
# should change
sm2.model.layers[-2].get_weights()[0][0,0]

-0.13824672

In [36]:
sm2.body_unfreeze()

In [37]:
sm2.save("/tmp/bpnet/model.pkl")

In [38]:
sm2.model.fit(seq, y)

Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 3.2015


In [39]:
sm2.save("/tmp/bpnet/model.pkl")

In [40]:
sm2.model.layers[1].get_weights()[0][0,0,0]

0.0054996004

In [41]:
# should change
sm2.model.layers[-2].get_weights()[0][0,0]

-0.1379275

In [42]:
sm2.save("/tmp/bpnet/model.pkl")

In [36]:
sm2.contrib_score(seq, 'output/pre-act')

array([[[ 3.8588408e-04,  2.7391803e-04,  8.4820658e-04,  1.2722018e-03],
        [-1.3604771e-03,  2.3850938e-03, -7.4564497e-04,  2.0508526e-03],
        [ 2.2248509e-03,  2.0788636e-03, -1.8977026e-03,  3.3240523e-03],
        ...,
        [ 1.7088703e-03,  2.6095244e-03,  4.7669960e-03,  1.0949221e-03],
        [ 2.2152655e-03,  4.3081371e-03,  4.0027304e-03,  8.4675843e-04],
        [ 2.0314781e-03,  3.4443170e-03,  3.5718894e-03,  1.7870904e-03]],

       [[ 2.7162782e-03,  6.8979356e-03, -1.8092996e-03, -2.3163954e-04],
        [ 1.0247475e-03,  4.4665304e-03,  1.6380259e-04,  2.1012486e-03],
        [-1.2963709e-03,  5.5719023e-03, -2.8568322e-03,  1.3635390e-03],
        ...,
        [-2.2348817e-03,  2.8472657e-03,  2.0541921e-03,  5.3054118e-03],
        [ 2.1051660e-03,  3.6225095e-03,  2.8354418e-03,  7.7408645e-04],
        [ 4.6703634e-03, -5.5563834e-04,  4.3853885e-03,  1.2375131e-04]],

       [[-2.1673078e-03,  1.6409596e-03,  1.7898149e-03,  3.8492959e-05],
        

## Save and load the model

In [43]:
from bpnet.seqmodel import SeqModel

In [44]:
sm3 = SeqModel.load("/tmp/bpnet/model.pkl")

In [45]:
# create training data
seq = np.random.randn(1000, sm3.seqlen, 4)
y = np.random.randn(1000)

In [46]:
sm3.contrib_score(seq, 'output/pre-act', method='deeplift')

DeepExplain: running "deeplift" explanation method (5)
Model with multiple inputs:  True


array([[[-2.4607759e-03,  3.5791998e-03,  2.2437777e-03,  2.0557018e-03],
        [ 3.3896070e-04,  2.6858198e-03, -1.9567017e-04,  2.1934751e-03],
        [ 5.4664910e-04,  1.8699621e-03, -7.7750842e-04,  1.8365451e-03],
        ...,
        [-5.8233989e-03, -1.0194867e-02, -6.3576819e-03, -2.2207247e-03],
        [ 4.9880130e-04, -1.0373782e-02, -7.3578991e-03, -3.0928329e-03],
        [-2.8041037e-04, -7.4002035e-03, -7.5780614e-03, -6.5689450e-03]],

       [[-4.9347157e-04,  3.2473307e-03,  1.2413186e-03,  2.2372210e-03],
        [ 3.7656413e-05,  1.6998422e-03,  1.3202829e-03,  9.3072711e-04],
        [ 8.7262510e-04,  1.7990355e-03,  2.6262109e-03, -1.6352844e-03],
        ...,
        [-1.0124389e-03, -1.1588347e-02, -7.7547603e-03, -4.6587251e-03],
        [-2.3345433e-03, -9.7960383e-03, -8.1247743e-03, -1.3785800e-03],
        [ 8.6217269e-04, -1.1490820e-02, -6.8263928e-03, -3.6769314e-03]],

       [[-3.5974220e-04,  1.0640307e-03, -8.1345189e-04,  3.7399563e-03],
        

In [14]:
import tensorflow as tf

In [15]:
tf.__version__

'1.7.0'